2018/11/05

嘗試練習用python做網頁爬蟲，一開始先使用這個模組抓取一些網頁原始碼。

在這裡我隨意找了google作為範例，我們先用urlopen對網頁進行開啟，之後再用read將其作為字串讀出，

由於大部分有中文內容，所以對其進行utf-8解碼，使用decode語法。

In [2]:
from urllib.request import urlopen

html = urlopen('https://www.google.com.tw/').read().decode('utf-8')
print(html)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb7 in position 5437: invalid start byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb7 in position 5329: invalid start byte

不知道為何無法decode特定位置的編碼，所以我決定忽略它，加個 'ignore'

In [5]:
html = urlopen('https://www.google.com.tw/').read().decode('utf-8', 'ignore')

接著我想要擷取這個網頁的標題，在html的架構下，編碼通常被< title> < /title>給框住，

這時候使用re.search正則表達的語法，
    
title = < title>(.+?)< /title>

'.' 表示除了換行'\n'以外的任意字符，'+'表示一次以上，'?' 表示盡可能匹配成最少的字符

In [7]:
import re
from urllib.request import urlopen

html = urlopen('https://www.google.com.tw/').read().decode('UTF-8', 'ignore')
title = '<title>(.+?)</title>'

print(re.search(title, html))

<re.Match object; span=(251, 272), match='<title>Google</title>'>


我們擷取到了網頁標題 match='< title>Google< /title>' 。

html語法內還有一個表達段落的，

paragraph = < p>(.*?)< /p>

但大部分網站結果都不多，可能要找一些網路文章的網頁比較適合。

html語法中，表示超連結的為 href="url"，

In [8]:
import re
from urllib.request import urlopen
html = urlopen('https://www.google.com.tw').read().decode('UTF-8', 'ignore')
url = 'href="(.*?)"'
print(re.search(url, html))

<re.Match object; span=(5424, 5478), match='href="https://www.google.com.tw/imghp?hl=zh-TW&ta>


只有找一個google image的超連結。所以我們用re.findall語法來抓出所有連結，

In [9]:
import re
from urllib.request import urlopen
html = urlopen('https://www.google.com.tw').read().decode('UTF-8', 'ignore')
url = 'href="(.*?)"'
print(re.findall(url, html))

['https://www.google.com.tw/imghp?hl=zh-TW&tab=wi', 'https://maps.google.com.tw/maps?hl=zh-TW&tab=wl', 'https://play.google.com/?hl=zh-TW&tab=w8', 'https://www.youtube.com/?gl=TW&tab=w1', 'https://news.google.com.tw/nwshp?hl=zh-TW&tab=wn', 'https://mail.google.com/mail/?tab=wm', 'https://drive.google.com/?tab=wo', 'https://www.google.com.tw/intl/zh-TW/about/products?tab=wh', 'http://www.google.com.tw/history/optout?hl=zh-TW', '/preferences?hl=zh-TW', 'https://accounts.google.com/ServiceLogin?hl=zh-TW&passive=true&continue=https://www.google.com.tw/', '/advanced_search?hl=zh-TW&amp;authuser=0', '/language_tools?hl=zh-TW&amp;authuser=0', '/intl/zh-TW/ads/', 'http://www.google.com.tw/intl/zh-TW/services/', '/intl/zh-TW/about.html', 'https://www.google.com.tw/setprefdomain?prefdom=US&amp;sig=K_gvKMV9shuCWbP4h1zl7fsdqS5qU%3D', '/intl/zh-TW/policies/privacy/', '/intl/zh-TW/policies/terms/']


這就是google首頁所有超連結的網址。

2018/12/09

這次嘗試使用爬蟲抓取ptt上的網頁內容，但遇到一個問題，

ptt上的網址基本上有禁止爬蟲行為的防治方式，解決辦法是使用一個 User Agent 繞過去，

In [11]:
from urllib.request import Request, urlopen

http = Request('https://www.ptt.cc/bbs/Steam/index.html', headers={'User-Agent': 'Mozilla/5.0'})
web = urlopen(http).read().decode('utf-8', 'ignore')